In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import pymysql
from sqlalchemy import create_engine
import getpass
password = getpass.getpass()

from imblearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

········


In order to optimize our inventory, we would like to know which films will be rented next month and we are asked to create a model to predict it.

Instructions
Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features.
Read the data into a Pandas dataframe.
Analyze extracted features and transform them. You may need to encode some categorical variables, or scale numerical variables.
Create a query to get the list of films and a boolean indicating if it was rented last month. This would be our target variable.
Create a logistic regression model to predict this variable from the cleaned data.
Evaluate the results.

## Project definition

In [9]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

q = '''
SELECT *, "2005-5" AS rental_year_month
  FROM film as f
  JOIN (SELECT f.film_id, COUNT(*) times_rented
		  FROM rental as r
		  JOIN inventory as i
			ON i.inventory_id = r.inventory_id
		  JOIN film as f
			ON f.film_id = i.film_id
		 WHERE SUBSTRING(r.rental_date, 1, 7) = "2005-05"
		 GROUP BY f.film_id) as q
	ON q.film_id = f.film_id
 ORDER BY q.times_rented DESC;
'''

data = pd.read_sql_query(q, engine)
data.head(2)

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update,film_id,times_rented,rental_year_month
0,103,BUCKET BROTHERHOOD,A Amazing Display of a Girl And a Womanizer wh...,2006,1,None,7,4.99,133,27.99,PG,"Commentaries,Deleted Scenes",2006-02-15 05:03:42,103,5,2005-5
1,450,IDOLS SNATCHERS,A Insightful Drama of a Car And a Composer who...,2006,1,None,5,2.99,84,29.99,NC-17,Trailers,2006-02-15 05:03:42,450,5,2005-5


In [10]:
data.shape

(686, 16)